# QPE data check

A quick notebook to check the domain and nodata values for all QPE data we want to support.

## Python requirements

- matplotlib
- numpy
- pandas
- requests
- rasterio
- tabulate

In [1]:
from dataclasses import dataclass
import gzip
import os
from typing import Optional

import requests

@dataclass(frozen=True)
class Download:
    name: str
    interval: str
    pass_number: int
    location: Optional[str]
    url: str
    path: str

names = [
    "MultiSensor_QPE_01H_Pass1",
    "MultiSensor_QPE_01H_Pass2",
    "MultiSensor_QPE_24H_Pass2",
]
locations = ["", "ALASKA", "CARIB", "GUAM", "HAWAII"]
server = "https://mrms.ncep.noaa.gov"
data_directory = "data"

os.makedirs(data_directory, exist_ok=True)

downloads = list()
for name in names:
    file_name = f"MRMS_{name}.latest.grib2.gz"
    parts = name.split("_")
    interval = parts[-2] 
    pass_number = parts[-1][-1]
    for location in locations:
        path = "/".join(filter(None, ["data", "2D", location, name, file_name]))
        url = f"{server}/{path}"
        outfile = os.path.join(data_directory, "_".join(filter(None, [name, location])) + ".grib2")
        print(f"Downloading {url} to {outfile}")
        response = requests.get(url)
        with open(outfile, "wb") as f:
            f.write(gzip.decompress(response.content))
        downloads.append(Download(name=name, interval=interval, pass_number=pass_number, location=location or None, url=url, path=outfile))

In [2]:
import numpy
import rasterio
import rasterio.plot
from pandas import DataFrame

rows = []
for download in downloads:
    with rasterio.open(download.path) as dataset:
        data = dataset.read(1)
        unique_negative_values = numpy.unique(data[data < 0])
        min = numpy.min(data[data >= 0])
        max = numpy.max(data[data >= 0])
        rows.append([download.name, download.interval, download.pass_number, download.location or "CONUS", min, max, [str(v) for v in unique_negative_values], dataset.nodatavals])

data_frame = DataFrame(rows, columns=["Name", "Interval", "Pass number", "Location", "Min", "Max", "Unique negative values", "Nodata values"])

In [3]:
import tabulate
print(tabulate.tabulate(data_frame, headers="keys", tablefmt="github"))

|    | Name                      | Interval   |   Pass number | Location   |   Min |   Max | Unique negative values   | Nodata values   |
|----|---------------------------|------------|---------------|------------|-------|-------|--------------------------|-----------------|
|  0 | MultiSensor_QPE_01H_Pass1 | 01H        |             1 | CONUS      |     0 |  82.8 | []                       | (None,)         |
|  1 | MultiSensor_QPE_01H_Pass1 | 01H        |             1 | ALASKA     |     0 |  13.5 | []                       | (None,)         |
|  2 | MultiSensor_QPE_01H_Pass1 | 01H        |             1 | CARIB      |     0 |  15.1 | ['-3.0']                 | (None,)         |
|  3 | MultiSensor_QPE_01H_Pass1 | 01H        |             1 | GUAM       |     0 |  21.6 | ['-3.0']                 | (None,)         |
|  4 | MultiSensor_QPE_01H_Pass1 | 01H        |             1 | HAWAII     |     0 |   6.4 | []                       | (None,)         |
|  5 | MultiSensor_QPE_01H_Pass2 |